<i>Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.</i>
<br><br>
# SVD Hyperparameter Tuning with Azure Machine Learning

In this notebook, we show how to tune the hyperparameters of a matrix factorization algorithm by utilizing **Azure Machine Learning service** ([AzureML](https://azure.microsoft.com/en-us/services/machine-learning-service/)) in the context of movie recommendations. To use AzureML you will need an Azure subscription. We use the SVD algorithm from the Surprise library.

We present the overall process of utilizing AzureML by demonstrating some key steps while avoiding too much detail. 

For more details about the **SVD** algorithm:
* [Surprise SVD deep-dive notebook](../02_model/surprise_svd_deep_dive.ipynb)
* [Original paper](http://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf)
* [Surprise homepage](https://surprise.readthedocs.io/en/stable/)
  
Regarding **AzureML**, please refer to:
* [Quickstart notebook](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python)
* [Hyperdrive](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters)

### 1. Global Settings

In [ ]:
import sys
sys.path.append("../../")
import time
import os
import json
import surprise
import datetime as dt
import papermill as pm
import numpy as np
import pandas as pd
import shutil
%matplotlib inline
import matplotlib.pyplot as plt
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import rmse, precision_at_k, ndcg_at_k
from reco_utils.recommender.surprise.surprise_utils import compute_rating_predictions, compute_ranking_predictions

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

import azureml as aml
import azureml.widgets
import azureml.train.hyperdrive as hd

print("Azure ML SDK Version:", aml.core.VERSION)

We assume that an AzureML workspace has already been created. For instructions how to do this, see [here](README.md).

In [ ]:
# AzureML workspace info. Note, will look up "aml_config\config.json" first, then fall back to using this
SUBSCRIPTION_ID = '<subscription-id>'
RESOURCE_GROUP  = '<resource-group>'
WORKSPACE_NAME  = '<workspace-name>'

# Connect to a workspace
try:
    ws = aml.core.Workspace.from_config()
except aml.exceptions.UserErrorException:
    try:
        ws = aml.core.Workspace(
            subscription_id=SUBSCRIPTION_ID,
            resource_group=RESOURCE_GROUP,
            workspace_name=WORKSPACE_NAME
        )
        ws.write_config()
    except aml.exceptions.AuthenticationException:
        ws = None

if ws is None:
    raise ValueError(
        """Cannot access the AzureML workspace w/ the config info provided.
        Please check if you entered the correct id, group name and workspace name"""
    )
else:
    print("AzureML workspace name: ", ws.name)

From the following cells, we
1. Create a *remote compute target* (cpu_cluster) if it does not exist already,
2. Mount a *data store* and upload the training set, and
3. Run a hyperparameter tuning experiment.

### 2. Create a Remote Compute Target

We create an AI Compute for our remote compute target. The script will load the cluster if it already exists. You can look at [this document](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets) to learn more about setting up a *compute target*.

> Note: we create a low priority cluster to save costs.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Remote compute (cluster) configuration. If you want to save costs decrease these.
# Each standard_D2_V2 VM has 2 vCPUs, 7GB memory, 100GB SSD storage

VM_SIZE = 'STANDARD_D2_V2'
VM_PRIORITY = 'dedicated' #'lowpriority'
# Cluster nodes
MIN_NODES = 8 #0
MAX_NODES = 8

# Choose a name for your CPU cluster
cpu_cluster_name = "cpuclustersvd"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=VM_SIZE, 
                                                           min_nodes=MIN_NODES, 
                                                           vm_priority=VM_PRIORITY,
                                                           max_nodes=MAX_NODES)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(cpu_cluster.status.serialize())

### 3. Prepare Dataset
1. Download data and split into training and testing sets
2. Upload the training set to the default **blob storage** of the workspace.

In [ ]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

In [ ]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

data.head()

In [ ]:
train, validation, test = python_random_split(data, ratio=[0.7, 0.15, 0.15], seed=42)

In [ ]:
DATA_DIR = 'aml_data'
os.makedirs(DATA_DIR, exist_ok=True)

TRAIN_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_train.pkl"
train.to_pickle(os.path.join(DATA_DIR, TRAIN_FILE_NAME))

VAL_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_val.pkl"
validation.to_pickle(os.path.join(DATA_DIR, VAL_FILE_NAME))

TEST_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_test.pkl"
test.to_pickle(os.path.join(DATA_DIR, TEST_FILE_NAME))

# Note, all the files under DATA_DIR will be uploaded to the data store
ds = ws.get_default_datastore()
ds.upload(
    src_dir=DATA_DIR,
    target_path='data',
    overwrite=True,
    show_progress=True
)

### 4. Prepare Hyperparameter Tuning 

We also prepare a training script [svd_training.py](../../reco_utils/azureml/svd_training.py) for the hyperparameter tuning, which will log our target metrics such as [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation) and/or [NDCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) to AzureML experiment so that we can track the metrics and optimize the primary metric via **hyperdrive**.

In [ ]:
SCRIPT_DIR = 'aml_script'

# Clean-up scripts if already exists
shutil.rmtree(SCRIPT_DIR, ignore_errors=True)

# Copy scripts to SCRIPT_DIR temporarly
shutil.copytree(os.path.join('..', '..', 'reco_utils'), os.path.join(SCRIPT_DIR, 'reco_utils'))

ENTRY_SCRIPT_NAME = 'reco_utils/azureml/svd_training.py'

Now we define a search space for the hyperparameters. All the parameter values will be passed to our training script.

We specify the output directory as ./outputs. The outputs directory is specially treated by Azure ML in that all the content in this directory gets uploaded to the workspace as part of the run history. The files written to this directory are therefore accessible even once the remote run is over. In the training script (svd_training.py), we use the output directory for saving the trained models. 

AzureML hyperdrive provides `RandomParameterSampling`, `GridParameterSampling`, and `BayesianParameterSampling`. Details about each approach are beyond the scope of this notebook and can be found in [Azure doc](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters). Here, we use the Bayesian sampling.

In [ ]:
EXP_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_svd_model"
PRIMARY_METRIC = 'precision_at_k'
RATING_METRICS = ['rmse']
RANKING_METRICS = ['precision_at_k', 'ndcg_at_k']  
USERCOL = 'userID'
ITEMCOL = 'itemID'
RECOMMEND_SEEN = False
K = 10
RANDOM_STATE = 0
VERBOSE = True
NUM_EPOCHS = 30
BIASED = True

script_params = {
    '--datastore': ds.as_mount(),
    '--train-datapath': "data/" + TRAIN_FILE_NAME,
    '--validation-datapath': "data/" + VAL_FILE_NAME,
    '--output_dir': './outputs',
    '--surprise-reader': 'ml-100k',
    '--rating-metrics': RATING_METRICS,
    '--ranking-metrics': RANKING_METRICS,
    '--usercol': USERCOL,
    '--itemcol': ITEMCOL,
    '--k': str(K),
    '--random-state': str(RANDOM_STATE),
    '--epochs': str(NUM_EPOCHS),
}

if BIASED:
    script_params['--biased'] = ''
if VERBOSE:
    script_params['--verbose'] = ''
if RECOMMEND_SEEN:
    script_params['--recommend-seen'] = ''
    
# hyperparameters search space
# We do not set 'lr_all' and 'reg_all' because they will be overwritten by the other lr_ and reg_ parameters

hyper_params = {
    '--n_factors': hd.choice(10, 50, 100, 150, 200),
    '--init_mean': hd.uniform(-0.5, 0.5),
    '--init_std_dev': hd.uniform(0.01, 0.2),
    '--lr_bu': hd.uniform(1e-6, 0.1), 
    '--lr_bi': hd.uniform(1e-6, 0.1), 
    '--lr_pu': hd.uniform(1e-6, 0.1), 
    '--lr_qi': hd.uniform(1e-6, 0.1), 
    '--reg_bu': hd.uniform(1e-6, 1),
    '--reg_bi': hd.uniform(1e-6, 1), 
    '--reg_pu': hd.uniform(1e-6, 1), 
    '--reg_qi': hd.uniform(1e-6, 1)
}

# Note, BayesianParameterSampling only support choice, uniform, and quniform
ps = hd.BayesianParameterSampling(hyper_params)

Once you submit the experiment, you can see the progress from the notebook by using `azureml.widgets.RunDetails`. You can directly check the details from the Azure portal as well. To get the link, run `run.get_portal_url()`.

For RandomSampling, you can use early termnination policy
```
policy = hd.BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=3)
```

> Since we will do hyperparameter tuning, we create a `HyperDriveRunConfig` and pass it to the experiment object. If you already know what hyperparameters to use and still want to utilize AzureML for other purposes (e.g. model management), you can set the hyperparameter values directly to `script_params` and run the experiment, `run = exp.submit(est)`, instead.  

In [ ]:
# Hyperdrive experimentation configuration
MAX_TOTAL_RUNS = 100  # Number of runs (training-and-evaluation) to search for the best hyperparameters. 
MAX_CONCURRENT_RUNS = 8

est = azureml.train.estimator.Estimator(
    source_directory=SCRIPT_DIR,
    entry_script=ENTRY_SCRIPT_NAME,
    script_params=script_params,
    compute_target=cpu_cluster,
    conda_packages=['pandas', 'scikit-learn'],
    pip_packages=['scikit-surprise', 'psutil']
)

hd_config = hd.HyperDriveRunConfig(
    estimator=est, 
    hyperparameter_sampling=ps,
    primary_metric_name=PRIMARY_METRIC,
    primary_metric_goal=hd.PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=MAX_TOTAL_RUNS,
    max_concurrent_runs=MAX_CONCURRENT_RUNS
)

### 5. Execute Runs in AzureML

In [ ]:
# Create an experiment to track the runs in the workspace
exp = aml.core.Experiment(workspace=ws, name=EXP_NAME)
run = exp.submit(config=hd_config)

azureml.widgets.RunDetails(run).show()
run.wait_for_completion(show_output=True)

You can see the experiment progress from this notebook by using `azureml.widgets.RunDetails(hd_run).show()` or check from the Azure portal with the url link you can get by running `hd_run.get_portal_url()`.
To load an existing Hyperdrive run, use `hd_run = hd.HyperDriveRun(exp, <user-run-id>, hyperdrive_run_config=hd_run_config)`. You also can cancel a run with `hd_run.cancel()`.
![](https://recodatasets.blob.core.windows.net/images/svd_hyperdrive1.PNG)
![](https://recodatasets.blob.core.windows.net/images/svd_hyperdrive2.PNG)


### 6. Download Logs

In [ ]:
parent_start_time = run.get_details()['startTimeUtc']
LOG_DIR = os.path.join('./log', parent_start_time)
CHILD_LOG_DIR = os.path.join(LOG_DIR, 'child_runs')
if not os.path.isdir(LOG_DIR):
    os.makedirs(LOG_DIR)
if not os.path.isdir(CHILD_LOG_DIR):
    os.makedirs(CHILD_LOG_DIR)

In [ ]:
def dump_log(local_path, file_name, log_info):
    with open(os.path.join(local_path, file_name), 'w') as fp:
        json.dump(log_info, fp)

parent_run_id = run.get_details()['runId']
dump_log(LOG_DIR, 'parent_run_details_' + parent_run_id + '.json', run.get_details())
dump_log(LOG_DIR, 'parent_run_metrics_' + parent_run_id + '.json', run.get_metrics())
for child_run in run.get_children():
    child_run_details = child_run.get_details()
    child_run_id = child_run_details['runId']
    dump_log(CHILD_LOG_DIR, 'child_run_details_' + child_run_id + '.json', child_run_details)
    dump_log(CHILD_LOG_DIR, 'child_run_metrics_' + child_run_id + '.json', child_run.get_metrics())

### 7. Check Results

In [ ]:
# Get best run and printout metrics
best_run = run.get_best_run_by_primary_metric()

best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

In [ ]:
best_run_metrics

In [ ]:
print(" ".join(parameter_values))

Now evaluate the metrics on the test data. To do this, get the SVD model that was saved as model.dump in the training script.

In [ ]:
os.makedirs('aml_model', exist_ok=True)
best_run.download_file('outputs/model.dump', output_file_path='aml_model/')

In [ ]:
svd = surprise.dump.load('aml_model/model.dump')[1]

In [ ]:
test_results = {}
predictions = compute_rating_predictions(svd, test, usercol="userID", itemcol="itemID")
for metric in RATING_METRICS:
    test_results[metric] = eval(metric)(test, predictions)

all_predictions = compute_ranking_predictions(svd, train, usercol="userID", itemcol="itemID", remove_seen=RECOMMEND_SEEN)
for metric in RANKING_METRICS:
    test_results[metric] = eval(metric)(test, all_predictions, col_prediction='prediction', k=K)

print(test_results)

### 8. Save Results

In [ ]:
def get_children_metric_endtime(run, metric_name, report_extra=True):
    """Get the start/end time and primary metric of each child run.
    """
    column_names = ["parent_run_id", 
                    "child_run_id", 
                    metric_name,
                    "parent_start_time",
                    "parent_end_time",
                    "child_start_time", 
                    "child_end_time"]
    if report_extra:
        extra_column_names = ["main_script_start_time",
                              "main_script_end_time",
                              "data_loading_start_time",
                              "data_loading_end_time",
                              "train_script_start_time",
                              "train_script_end_time"]
        column_names += extra_column_names
    df = pd.DataFrame(columns=column_names)
    run_details = run.get_details()
    for idx, child_run in enumerate(run.get_children()):
        child_run_details = child_run.get_details()
        child_run_metrics = child_run.get_metrics()
        if child_run_details["properties"]["azureml.runsource"] == "experiment":
            parent_run_id = run_details["runId"]
            child_run_id = child_run_details["runId"]
            primary_metric = child_run_metrics[metric_name]
            parent_start_time = run_details["startTimeUtc"]
            parent_end_time = run_details["endTimeUtc"]
            child_start_time = child_run_details["startTimeUtc"]
            child_end_time = child_run_details["endTimeUtc"]
            row = [parent_run_id, 
                   child_run_id, 
                   primary_metric,
                   parent_start_time,
                   parent_end_time,
                   child_start_time, 
                   child_end_time]
            if report_extra:
                for timestamp_name in extra_column_names:
                    row.append(child_run_metrics[timestamp_name])
                    
            df.loc[idx] = row

    return df

results = get_children_metric_endtime(run, PRIMARY_METRIC)

In [ ]:
results.head()

In [ ]:
TIMESTAMP_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"
results["child_end_time"] = pd.to_datetime(results["child_end_time"], format=TIMESTAMP_FORMAT)
parent_start_time = dt.datetime.strptime(parent_start_time, TIMESTAMP_FORMAT)
results["time_since_start"] = results.apply(lambda row: (row["child_end_time"] - parent_start_time).total_seconds(), axis=1)
results.sort_values(by=["time_since_start"], inplace=True)
results["best_metric_so_far"] = results[metric_name].cummax()

results['best_metric'] = best_run_metrics[PRIMARY_METRIC]
results['best_parameters'] = " ".join(parameter_values)
results['test_metric'] = test_results[PRIMARY_METRIC]

In [ ]:
results["child_start_time"] = pd.to_datetime(results["child_start_time"], format=TIMESTAMP_FORMAT)
results["parent_start_time"] = pd.to_datetime(results["parent_start_time"], format=TIMESTAMP_FORMAT)
results["parent_end_time"] = pd.to_datetime(results["parent_end_time"], format=TIMESTAMP_FORMAT)

results["total_time"] = (results.iloc[0]["parent_end_time"] - results.iloc[0]["parent_start_time"]).total_seconds()
results["execution_time"] = (max(results["child_end_time"]) - min(results["child_start_time"])).total_seconds()
results["deployment_time"] = (min(results["child_start_time"]) - results.iloc[0]["parent_start_time"]).total_seconds()
results["clean_up_time"] = (results.iloc[0]["parent_end_time"] - max(results["child_end_time"])).total_seconds()

In [ ]:
# Save results to csv file
result_path = os.path.join(LOG_DIR, "results.csv")
results.to_csv(result_path, index=False)

### 9. Result Visualization

In [ ]:
# Plot accuracy vs. time
plt.xlabel("time (s)")
plt.ylabel("precision_at_k")
plt.xlim((0, max(results["time_since_start"])))
plt.ylim((0, 0.1))
plt.title("HyperDrive")
plt.step(results["time_since_start"], results["best_metric_so_far"])
plt.show()

In [ ]:
def stacked_bar(data, series_labels, category_labels=None, 
                show_values=False, value_format="{}", y_label=None, 
                grid=True, reverse=False):
    """Plots a stacked bar chart with the data and labels provided.
    Copied from https://stackoverflow.com/questions/44309507/stacked-bar-plot-using-matplotlib

    Keyword arguments:
    data            -- 2-dimensional numpy array or nested list
                       containing data for each series in rows
    series_labels   -- list of series labels (these appear in
                       the legend)
    category_labels -- list of category labels (these appear
                       on the x-axis)
    show_values     -- If True then numeric value labels will 
                       be shown on each bar
    value_format    -- Format string for numeric value labels
                       (default is "{}")
    y_label         -- Label for y-axis (str)
    grid            -- If True display grid
    reverse         -- If True reverse the order that the
                       series are displayed (left-to-right
                       or right-to-left)
    """

    ny = len(data[0])
    ind = list(range(ny))

    axes = []
    cum_size = np.zeros(ny)

    data = np.array(data)

    if reverse:
        data = np.flip(data, axis=1)
        category_labels = reversed(category_labels)

    for i, row_data in enumerate(data):
        axes.append(plt.bar(ind, row_data, bottom=cum_size, 
                            label=series_labels[i]))
        cum_size += row_data

    if category_labels:
        plt.xticks(ind, category_labels)

    if y_label:
        plt.ylabel(y_label)

    plt.legend()

    if grid:
        plt.grid()

    if show_values:
        for axis in axes:
            for bar in axis:
                w, h = bar.get_width(), bar.get_height()
                plt.text(bar.get_x() + w/2, bar.get_y() + h/2, 
                         value_format.format(h), ha="center", 
                         va="center")

In [ ]:
plt.figure(figsize=(6, 4))

series_labels = ['deployment', 'execution', 'clean-up']

data = [
    [results.iloc[0]['deployment_time']], 
    [results.iloc[0]['execution_time']], 
    [results.iloc[0]['clean_up_time']]
]

category_labels = ['HyperDrive']

stacked_bar(
    data, 
    series_labels, 
    category_labels=category_labels, 
    show_values=True, 
    value_format="{:.1f}",
    y_label="Quantity (units)",
    grid=False
)

#plt.savefig('bar.png')
plt.show()

In [ ]:
try:
    shutil.rmtree(SCRIPT_DIR)
    shutil.rmtree(DATA_DIR)
except (PermissionError, FileNotFoundError):
    pass

### 9. Concluding Remarks

We showed how to tune **all** the hyperparameters accepted by Surprise SVD simultaneously, by utilizing the Azure Machine Learning service. 
For example, training and evaluation of a single SVD model takes about 50 seconds on the 100k MovieLens data on a Standard D2_V2 VM. Searching through 100 different combinations of hyperparameters sequentially would take about 80 minutes whereas this notebook took less than half that. With AzureML, one can easily specify the size of the cluster according to the problem at hand and use Bayesian sampling to navigate efficiently through a large space of hyperparameters.

### References

* [Matrix factorization algorithms in Surprise](https://surprise.readthedocs.io/en/stable/matrix_factorization.html) 
* [Surprise SVD deep-dive notebook](../02_model/surprise_svd_deep_dive.ipynb)
* [Fine-tune natural language processing models using Azure Machine Learning service](https://azure.microsoft.com/en-us/blog/fine-tune-natural-language-processing-models-using-azure-machine-learning-service/)
* [Training, hyperparameter tune, and deploy with TensorFlow](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training-with-deep-learning/train-hyperparameter-tune-deploy-with-tensorflow/train-hyperparameter-tune-deploy-with-tensorflow.ipynb)
